In [1]:
#basecode what I showed on Monday.  , go to end for new code. 
code = '''import os    
import numpy as np
import numpy.ma as ma   
#import awkward as ak 
from tqdm import tqdm 
import torch 
import torch_geometric as tg  
import torch_geometric 
from torch_geometric.data import Data 
#import torch 
from torch_geometric.data import InMemoryDataset, download_url  
import torch_geometric.transforms as T 

#NOTE: NEW 2/20/23      
from typing import List, Union     

from torch_geometric.data import Data, HeteroData 
from torch_geometric.data.datapipes import functional_transform
from torch_geometric.transforms import BaseTransform       
torch.cuda.empty_cache()  
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import GraphConv
from torch_geometric.nn import global_mean_pool
from torch_geometric.nn.norm import GraphNorm, BatchNorm 
from torch.utils.data import random_split 
from torch_geometric.loader import DataLoader 

class MyOwnDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None, pre_filter=None):
        super().__init__(root, transform, pre_transform, pre_filter)
        self.load(self.processed_paths[0])
        # For PyG<2.4:
        # self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return ['some_file_1', 'some_file_2']

    @property
    def processed_file_names(self):
        return ['data.pt']

    def process(self):
        # Read data into huge `Data` list.
        data_list = None

        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]

        self.save(data_list, self.processed_paths[0]) 

#root = '/hpc/group/vossenlab/kam264/L_imass3'
#root = '/hpc/group/vossenlab/kam264/onlylambda_25k' 
#root = '/hpc/group/vossenlab/kam264/Lambda_13_16'
#root = '/hpc/group/vossenlab/kam264/Lambda_1all' 

batch_size = 16 
LR =1e-2
torch.cuda.empty_cache()   

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels): 
        super(GCN, self).__init__()
#         torch.manual_seed(12345)
        self.conv1 = GCNConv(in_channels, hidden_channels)#.jittable() #NOTE: NEEDED FOR DEPLOYMENT IN CMAKE
        self.conv2 = GCNConv(hidden_channels, hidden_channels)#.jittable()
        #self.block2 = nn.DataParallel(self.block2)
        #self.conv2 = torch.nn.DataParallel(self.conv2) #this was trying the parallization thing. 
        self.conv3 = GCNConv(hidden_channels, hidden_channels)#.jittable()
        #self.conv3 = torch.nn.DataParallel(self.conv3)
        self.lin1 = Linear(hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, hidden_channels)
        self.lin3 = Linear(hidden_channels, out_channels)
        self.bn1 = torch_geometric.nn.norm.GraphNorm(hidden_channels)
        self.bn2 = torch_geometric.nn.norm.GraphNorm(hidden_channels)
        self.bn3 = torch_geometric.nn.norm.GraphNorm(hidden_channels)

    def forward(self, x, edge_index, batch):  
        
        x = self.conv1(x, edge_index) #input layer                             
                                                      
        x = self.bn1(x) #normalize it                                          

        x = x.relu() #activation                                               
        x = self.conv2(x, edge_index)
        x = self.bn2(x)
        x = x.relu()
#         print("x.relu() = ",x)  
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv3(x, edge_index)
        x = self.bn3(x)
#         # 2. Readout layer                                                   
        x = global_mean_pool(x, batch)
        # 3. Apply a final classifier                                          
        x = F.dropout(x, p=0.5, training=self.training) #for overfittin        
        x = self.lin3(x) 
 
        return x
def RMSELoss(out,y):
    return torch.sqrt(torch.mean((out-y)**2))
def train():
    model.train() #initailize the model                                                                                                                                                                                                        
    #for i, data in tqdm(enumerate(train_loader)): #perhaps tqdm(enumerate(train_loader)), i is index, data jsut moves through all the dtaa in trainingg                                                                                      
    for i,data in enumerate(train_loader):
        data = data.to(device) #switch to GPU                                                                                                                                                                                                 
        optimizer.zero_grad() #                                                                                                                                                                                                               
        out = model(data.x, data.edge_index, data.batch).to(device)  # Perform a single forward pass                                                                                                                                          
        yy = []
        for j in range(0,len(out)):
            yy+= [[data.y[j].item()]]

        yy = torch.tensor(yy).to(device) 
        #print(out)
        #print(yy)
        loss = losss(out, yy).to(device) #compute the loss  
        #print(loss)
        loss.backward() #get the gradients.                                                                                                                                                                                                   
        optimizer.step() 
def test(loader): 
    length = len(loader.dataset)
    model.eval() #evaluate teh model.                                                                                                                                                                                                         

    #mse_tot = []                                                                                                                                                                                                                             
    mse_total = 0
    mse_pi = 0
    mse_p = 0
    #r                                                                                                                                                                                                                                        
    #for data in tqdm(loader):  # Iterate in batches over the training/test dataset.                                                                                                                                                          
    for data in loader:
        data = data.to(device) #put to GPU                                                                                                                                                                                                    
        out = model(data.x, data.edge_index, data.batch).to(device) #evalueate                                                                                                                                                                
        #this and the for loop is converting data.y to a tensor in the same shape as out rows and 2 columns first is y_pion second is y_proton                                                                                                
        yy = []
        for j in range(0,len(out)):
            yy+= [[data.y[j].item()]]
        yy = torch.tensor(yy).to(device) 
        loss = losss(out, yy).cpu() #getting teh loss function                                                                                                                                                                                
        mse_total+=loss.item() #getting the mse (total)                                                                                                                                                                                       

    return mse_total/length 
def print_out():
    model.eval() #initailize the model                                                                                                                                                                                                        
    #for i, data in tqdm(enumerate(train_loader)): #perhaps tqdm(enumerate(train_loader)), i is index, data jsut moves through all the dtaa in trainingg                                                                                      
    outt= []
    for i,data in enumerate(test_loader):
        data = data.to(device) #switch to GPU                                                                                                                                                                                                 
        optimizer.zero_grad() #                                                                                                                                                                                                               
        out = model(data.x, data.edge_index, data.batch).to(device)  # Perform a single forward pass                                                                                                                                          
        out = out.cpu()

        outt+=[[out.detach().numpy()]]
    return outt

def print_outb():
    model.eval() #initailize the model                                                                                                                                                                                                        
    #for i, data in tqdm(enumerate(train_loader)): #perhaps tqdm(enumerate(train_loader)), i is index, data jsut moves through all the dtaa in trainingg      

    outt= [] 
    #for i,data in enumerate(dataset):
    for i,data in enumerate(test_loader):
        data = data.to(device) #switch to GPU                                                                                                                                                                                                 
        optimizer.zero_grad() #                                                                                                                                                                                                               
        out = model(data.x, data.edge_index, data.batch).to(device)  # Perform a single forward pass                                                                                                                                          
        out = out.cpu()

        neww = []
        for j in range(len(data)):
            neww.append(out[j].item() -data.elc[j][0]) 
        outt+=[[neww]]
    return outt  
#model = GCN(dataset.num_node_features,64,2)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') 
#devicee = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu') 
print("Device = ",device) 

print("DEBUGGING: torch.cuda.is_available() = ",torch.cuda.is_available())
train_metrics = {'mse':[], "rmse":[] } 
vall_metrics = {'mse':[], "rmse":[] }

for iii in range(1):    
    if iii>0: 
        #del dataset
        del train_loader  
        del val_loader 
    if iii==0:
        root = '/hpc/group/vossenlab/kam264/Qmom0'
    if iii==1:
        root = '/hpc/group/vossenlab/kam264/Qmom1'
    if iii==2:
        root = '/hpc/group/vossenlab/kam264/Qmom2'
    if iii==3:
        root = '/hpc/group/vossenlab/kam264/Qmom3'   
    dataset = MyOwnDataset(
            root,
            transform=None, #T.Compose([T.ToUndirected(),T.KNNGraph(k=6)]),
            pre_transform=None,
            pre_filter=None
        ) 
    dataset
    model = GCN(dataset.num_node_features,64,2)
    fracs = [0.9, 0.08, 0.02] #percent of dataset used for training testing and validatoin 80%,10%,10% #NOTE: SHOULD CHECK np.sum(fracs) == 1 and len(fracs)==3
    fracs = [torch.sum(torch.tensor(fracs[:idx])) for idx in range(1,len(fracs)+1)] #get the indexes for training ... parts to use. 
    #print(fracs)
    split1, split2 = [int(len(dataset)*frac) for frac in fracs[:-1]] 
    train_dataset = dataset[:split1]
    val_dataset = dataset[split1:split2]
    test_dataset = dataset[split2:]   
     
    print(f'Number of training graphs: {len(train_dataset)}')
    print(f'Number of validation graphs: {len(val_dataset)}') 

    from torch_geometric.loader import DataLoader 
    #from torch.utils.data import WeightedRandomSampler

    train_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=8, shuffle=True)#, drop_last=True)
    val_loader = DataLoader(val_dataset,  batch_size=batch_size,num_workers=8,  shuffle=True)
    model = GCN(dataset.num_node_features, dataset.num_classes, 1).to(device) #initiate the model, #2 is the number of outputs here is 2 as pion_z, proton_z 
    model = model.to(device)  
    del dataset
    del train_dataset
    del val_dataset
    del test_dataset 
    optimizer = torch.optim.Adam(model.parameters(), lr= LR)  
    
    losss = RMSELoss  
    nepochs = 800     
    
    for epoch in range(nepochs):  
  
        #print("BEFORE TRAIN()")                                                                                                                                                                                                                  
        train() 
        #print("BEFORE TEST(TRAIN_LOADER)")                                                                                                                                                                                                       
        #train_mse, train_rmse, train_mse_pi, train_rmse_pi, train_mse_p, train_rmse_p = test(train_loader)
        train_mse = test(train_loader)

        train_metrics['mse'].append(train_mse) 

        vall_mse =test(val_loader) 


        vall_metrics['mse'].append(vall_mse) 

        if epoch%9==0:
            print("Epoch ",epoch," Train mse: ",train_mse)
            print("Epoch ",epoch," Validation mse: ",vall_mse)
        if epoch==(nepochs-1):
            #PATH = '/work/clas12/users/mfmce/CLAS12_Lambda_resolution_REU_2023/model_best_auc.pt' 

            #a = print_out()
            #b=  print_outb() 
            print("Epoch ",epoch," Train mse: ",train_mse)
            print("Epoch ",epoch," Validation mse: ",vall_mse) 

root = '/hpc/group/vossenlab/kam264/Hmom4' 
#root = '/hpc/group/vossenlab/kam264/QuatorL_Last'
dataset = MyOwnDataset(
        root,
        transform=None, #T.Compose([T.ToUndirected(),T.KNNGraph(k=6)]),
        pre_transform=None,
        pre_filter=None
    )    

model.eval() #initailize the model                                                                                                                                                                                                        
#for i, data in tqdm(enumerate(train_loader)): #perhaps tqdm(enumerate(train_loader)), i is index, data jsut moves through all the dtaa in trainingg      

outt= []  
pi_mas = []
#for i,data in enumerate(dataset): 
for i,data in enumerate(dataset):
    data = data.to(device) #switch to GPU                                                                                                                                                                                                 
    #optimizer.zero_grad() #                                                                                                                                                                                                               
    out = model(data.x, data.edge_index, data.batch).to(device)  # Perform a single forward pass                                                                                                                                          
    out = out.cpu()
    #print(out) 
    #print(data.elc) 

   # neww = []
    #for j in range(len(data)):
        #neww.append(out[j].item() -data.elc[j][0])
     #   neww.append(out[j].item() -data.elc[j])
    #outt+=[[neww]]
    #outt+=[[out[0][0].item()-data.elc[0]]] 
    outt+=[[out[0][0].item()-data.elc[0]]]
    for j in range(0,int(len(data.imass))): 
        pi_mas.append(data.imass[j] ) 
#return outt  
#b= print_outbb() 
b=outt 

import matplotlib.pyplot as plt
pi_x = np.zeros(len(outt))
for i in range(len(outt)):
    pi_x[i] = outt[i][0]
pi_yy = [] 
#p_yy = [] 
#pi_x = outt
pi_rec_y = []
#p_rec_y = []
for i, data in enumerate(dataset):
    data = data#.to(device) #put to GPU
    #break
    jj= 0
    for j in range(0,int(len(data.y))): 
        pi_yy.append(data.y[j].item() -data.elc[j] ) 
      
    for j in range(len(data.rec)):
        pi_rec_y.append(data.rec[j]-data.elc[j])
       # p_rec_y.append(data.rec[j][0][1])

#plott_pi = np.zeros((len(pi_x),4))
#plott_p = np.zeros((len(p_x),2)) 
plott_pi = np.zeros((len(pi_x),3))
for i in range(len(pi_x)):
#for i in range(982):
    plott_pi[i][0] = pi_yy[i]
    plott_pi[i][1] = pi_x[i]# [0]
    #plott_pi[i][3] = pi_val[i]
   # plott_pi[i][2] = pi_rec_y[i] 
   # plott_p[i][0] = p_yy[i]
   # plott_p[i][1] = p_x[i]
    #plott_p[i][2]  = p_rec_y[i] 
for i in range(len(pi_x)): 
    plott_pi[i][2] = pi_rec_y[i] 
#plott_pi 
plt.figure(0) ##################
plt.title('location of Pion vertex') 
#plt.axvline(x = np.mean(pi_x), color = 'blue') 
#plt.axvline(x = np.mean(pi_yy), color = 'orange') 
plt.hist(x = plott_pi, histtype ='step', color = ['orange', 'blue', 'green'], bins = 500 ) 
#plt.hist(x = plott_pi, histtype ='step', bins = 500 )
#plt.legend(['Predicted', 'True', 'REC']) 
#plt.legend(['True', 'Predicted', 'REC']) 
plt.xlim((-10,15)) 
plt.ylabel('count') 
plt.xlabel('Location of Pion vertex (cm)')  
 #############################################################

plt.figure(1) #######################
pi_yy = np.array(pi_yy)
pi_x = np.array(pi_x)
pi_rec_y = np.array(pi_rec_y)#np.logical_and
indices_two = np.where(np.logical_and(pi_yy < -1.5, pi_yy > -2) ) 
#indices_two = np.where(pi_yy <-1.5 and pi_yy>-2 )
#print(indices_two)
#print(pi_x[indices_two])  

diff_x = [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
diff_rec  = [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
for j in range(1,24):
    indices = np.where(np.logical_and(pi_yy < -2+j/2, pi_yy > -2+j/2-.5) ) 
    dif_x =[ abs(pi_x[indices] - pi_yy[indices])]
    dif_rec = [abs(pi_rec_y[indices] - pi_yy[indices])]
    diff_x[j-1].append(dif_x)
    diff_rec[j-1].append(dif_rec)
mean_x = np.zeros(24)
mean_rec = np.zeros(24)
error_x = np.zeros(24)
error_rec = np.zeros(24)
for i in range(24): 
    mean_x[i] = np.nanmean(diff_x[i])
    mean_rec[i] = np.nanmean(diff_rec[i]) 
    error_x[i] = np.nanstd(diff_x[i])
    error_rec[i] = np.nanstd(diff_rec[i])
xxx = np.linspace(-2,10,24)
plt.scatter(xxx, mean_x)
plt.scatter(xxx, mean_rec, alpha = .5 )  
plt.errorbar(xxx, mean_x,  yerr=error_x, fmt="o")   
plt.errorbar(xxx, mean_rec,  yerr=error_rec, fmt="o", alpha = .5)
plt.title('absolute error based on pion vertex, orange is rec, blue is GNN')
plt.xlabel('Pion vertex (cm)') 
plt.ylabel('abolute error from True (cm)')
 #############################################################
plt.figure(2) ####################### 
diff_x = [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
diff_rec  = [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
for j in range(1,24):
    indices = np.where(np.logical_and(pi_yy < -2+j/2, pi_yy > -2+j/2-.5) ) 
    dif_x =[ (pi_x[indices] - pi_yy[indices])]
    dif_rec = [(pi_rec_y[indices] - pi_yy[indices])]
    diff_x[j-1].append(dif_x)
    diff_rec[j-1].append(dif_rec)
mean_x = np.zeros(24)
mean_rec = np.zeros(24)
error_x = np.zeros(24)
error_rec = np.zeros(24)
for i in range(24): 
    mean_x[i] = np.nanmean(diff_x[i])
    mean_rec[i] = np.nanmean(diff_rec[i]) 
    error_x[i] = np.nanstd(diff_x[i])
    error_rec[i] = np.nanstd(diff_rec[i])
xxx = np.linspace(-2,10,24)
plt.scatter(xxx, mean_x)
plt.scatter(xxx, mean_rec, alpha = .5 )  
plt.errorbar(xxx, mean_x,  yerr=error_x, fmt="o")   
plt.errorbar(xxx, mean_rec,  yerr=error_rec, fmt="o", alpha = .5)
plt.title(' error based on pion vertex, orange is rec, blue is GNN')
plt.xlabel('Pion vertex (cm)') 
plt.ylabel(' error from True (cm)')
 #############################################################

pi_mas = []
pi_LamT = []
#p_rec_y = []
for i, data in enumerate(dataset):
    data = data#.to(device) #put to GPU
    #break
    #print(data.LamT)
    for j in range(0,int(len(data.imass))): 
        pi_mas.append(data.imass[j]) 
        pi_LamT.append(data.LamT[j]) 
Total_signal = pi_LamT.count(1)
print('S/B')
print('Purity') 
print('Efficienty')
s_to_b_values = []
purity_values = []
efficiency_values = []
for j in range(0, 20):
    pi_LamTt = []
    #imass
    print('cut ',j/2)
    for i in range(len(pi_x)): 
        if pi_x[i]>j/2:
            #imass.append(pi_mas[i])
            pi_LamTt.append(pi_LamT[i])
    print(pi_LamTt.count(1))
    #print(pi_LamT.count(0))
    print(pi_LamTt.count(1)/pi_LamTt.count(0))
    print(pi_LamTt.count(1)/(len(pi_LamTt)))
    print(pi_LamTt.count(1)/(Total_signal))
    #print(pi_LamTt.count(1)/len(pi_LamTt))
    s_to_b = pi_LamTt.count(1)/pi_LamTt.count(0)
    purity = pi_LamTt.count(1)/(len(pi_LamTt))
    efficiency = pi_LamTt.count(1)/(Total_signal)
    s_to_b_values.append(s_to_b)
    purity_values.append(purity)
    efficiency_values.append(efficiency)
    print()
    plt.figure(3)
    plt.scatter(j/2, pi_LamTt.count(1)/pi_LamTt.count(0), color = 'blue')
    plt.xlabel('pion vetex cut')
    plt.ylabel('S/B') 
    plt.title('Graph of S/B as pion vertex cuts')
    
    plt.figure(4)
    plt.scatter(j/2, pi_LamTt.count(1)/(len(pi_LamTt)), color = 'red')
    plt.xlabel('pion vetex cut')
    plt.ylabel('Purity') 
    plt.title('Graph of Purity as pion vertex cuts')
                
    plt.figure(5)
    plt.scatter(j/2, pi_LamTt.count(1)/(Total_signal), color = 'purple') 
    plt.xlabel('pion vetex cut')
    plt.ylabel('Efficiency ') 
    plt.title('Graph of Efficiency  as pion vertex cuts')
    #print(len(pi_LamT))
    #plt.figure()
    #plt.title('cut'+str( j/2))  
    #plt.hist(imass, bins = 25)
##################################################################new stufff. 
pi_x = np.zeros(len(outt)) 
for i in range(len(outt)):
    pi_x[i] = outt[i][0]
pi_yy = [] 
#p_yy = [] 
#pi_x = outt
pi_rec_y = []
#p_rec_y = []
for i, data in enumerate(dataset):
    data = data#.to(device) #put to GPU
    jj= 0
    for j in range(0,int(len(data.y))): 
        pi_yy.append(data.y[j].item() -data.elc[j] ) 
      
    for j in range(len(data.rec)):
        pi_rec_y.append(data.rec[j]-data.elc[j])

plott_pi = np.zeros((len(pi_x),3))
for i in range(len(pi_x)):
    if pi_rec_y[i]>4:
        if pi_rec_y[i]<6:
            plott_pi[i][0] = pi_yy[i]
            plott_pi[i][1] = pi_x[i]# [0]

for i in range(len(pi_x)): 
    if pi_rec_y[i]>4:
        if pi_rec_y[i]<6:
            plott_pi[i][2] = pi_rec_y[i] 
#plott_pi 

plt.figure(7) #############################################################
pi_yy = np.array(pi_yy)
pi_x = np.array(pi_x)
pi_rec_y = np.array(pi_rec_y)#np.logical_and
indices_two = np.where(np.logical_and(pi_yy < -1.5, pi_yy > -2) ) 
#indices_two = np.where(pi_yy <-1.5 and pi_yy>-2 )
#print(indices_two)
#print(pi_x[indices_two])  

diff_x = [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
diff_rec  = [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
for j in range(1,24):
    indices = np.where(np.logical_and(pi_rec_y < -2+j/2, pi_rec_y > -2+j/2-.5) ) 
    dif_x =[ abs(pi_x[indices] - pi_yy[indices])]
    dif_rec = [abs(pi_rec_y[indices] - pi_yy[indices])]
    diff_x[j-1].append(dif_x)
    diff_rec[j-1].append(dif_rec)
mean_x = np.zeros(24)
mean_rec = np.zeros(24)
error_x = np.zeros(24)
error_rec = np.zeros(24)
for i in range(24): 
    mean_x[i] = np.nanmean(diff_x[i])
    mean_rec[i] = np.nanmean(diff_rec[i]) 
    error_x[i] = np.nanstd(diff_x[i])
    error_rec[i] = np.nanstd(diff_rec[i])
xxx = np.linspace(-2,10,24)
print('based on rec')
print(mean_x)
print(mean_rec)
print(error_x)
print(error_rec)
print()
plt.scatter(xxx, mean_x)
plt.scatter(xxx, mean_rec, alpha = .5 )  
plt.errorbar(xxx, mean_x,  yerr=error_x, fmt="o")   
plt.errorbar(xxx, mean_rec,  yerr=error_rec, fmt="o", alpha = .5)
plt.title('Error Based on rec Vertex')
plt.xlabel('Pion rec Vertex (cm)') 
plt.ylabel('Abolute Error from True (cm)')
#plt.savefig('Newstuff2.png')

plt.figure(8) #############################################################
pi_yy = np.array(pi_yy)
pi_x = np.array(pi_x)
pi_rec_y = np.array(pi_rec_y)#np.logical_and
indices_two = np.where(np.logical_and(pi_yy < -1.5, pi_yy > -2) ) 
#indices_two = np.where(pi_yy <-1.5 and pi_yy>-2 )
#print(indices_two)
#print(pi_x[indices_two])  

diff_x = [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
diff_rec  = [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
for j in range(1,24):
    indices = np.where(np.logical_and(pi_x < -2+j/2, pi_x > -2+j/2-.5) ) 
    dif_x =[ abs(pi_x[indices] - pi_yy[indices])]
    dif_rec = [abs(pi_rec_y[indices] - pi_yy[indices])]
    diff_x[j-1].append(dif_x)
    diff_rec[j-1].append(dif_rec)
mean_x = np.zeros(24)
mean_rec = np.zeros(24)
error_x = np.zeros(24)
error_rec = np.zeros(24)
for i in range(24): 
    mean_x[i] = np.nanmean(diff_x[i])
    mean_rec[i] = np.nanmean(diff_rec[i]) 
    error_x[i] = np.nanstd(diff_x[i])
    error_rec[i] = np.nanstd(diff_rec[i])
xxx = np.linspace(-2,10,24)
print('based on GNN')
print(mean_x)
print(mean_rec)
print(error_x)
print(error_rec)
print() 
plt.scatter(xxx, mean_x)
plt.scatter(xxx, mean_rec, alpha = .5 )  
plt.errorbar(xxx, mean_x,  yerr=error_x, fmt="o")   
plt.errorbar(xxx, mean_rec,  yerr=error_rec, fmt="o", alpha = .5)
plt.title('Error based on GNN vertex (abs)')
plt.xlabel('Pion GNN vertex (cm)') 
plt.ylabel('Abolute Error from True (cm)')
#plt.savefig('Newstuff3.png') 







outt= [] 
pi_momentum  = []
#for i,data in enumerate(dataset): 
for i,data in enumerate(dataset):
    data = data.to(device) #switch to GPU                                                                                                                                                                                                 
    #optimizer.zero_grad() #                                                                                                                                                                                                               
    #out = model(data.x, data.edge_index, data.batch).to(device)  # Perform a single forward pass                                                                                                                                          
    #out = out.cpu()
    #print(out) 
    #print(data.elc)  
   # neww = []
    #for j in range(len(data)):
        #neww.append(out[j].item() -data.elc[j][0])
     #   neww.append(out[j].item() -data.elc[j])
    #outt+=[[neww]]
    #outt+=[[out[0][0].item()-data.elc[0]]] 
    #outt+=[[out[0][0].item()-data.elc[0]]]
    for j in range(0,int(len(data.momentumt))): 
        pi_momentum.append(data.momentumt[j] )  
        
pi_momentum = np.array(pi_momentum) 
plt.figure(9) ####################### 
diff_x = [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
diff_rec  = [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
for j in range(1,20):
    indices = np.where(np.logical_and(pi_momentum < 0+j/4, pi_momentum > 0+j/4-.25) )  
    dif_x =[ abs(pi_x[indices] - pi_yy[indices])]
    dif_rec = [abs(pi_rec_y[indices] - pi_yy[indices])]
    diff_x[j-1].append(dif_x)
    diff_rec[j-1].append(dif_rec)
mean_x = np.zeros(20)
mean_rec = np.zeros(20)  
error_x = np.zeros(20)
error_rec = np.zeros(20)
for i in range(20): 
    mean_x[i] = np.nanmean(diff_x[i])
    mean_rec[i] = np.nanmean(diff_rec[i]) 
    error_x[i] = np.nanstd(diff_x[i])
    error_rec[i] = np.nanstd(diff_rec[i])
xxx = np.linspace(0,5,20) 
plt.scatter(xxx, mean_x)
plt.scatter(xxx, mean_rec, alpha = .5 )  
plt.errorbar(xxx, mean_x,  yerr=error_x, fmt="o")   
plt.errorbar(xxx, mean_rec,  yerr=error_rec, fmt="o", alpha = .5)
plt.title(' Error based on Momentum')
plt.xlabel('Pion momentum (cm)') 
plt.ylabel(' Error from True (cm)')  ##################################################


outt= []  
pi_mas = []
#for i,data in enumerate(dataset): 
for i,data in enumerate(dataset):
    data = data.to(device) #switch to GPU                                                                                                                                                                                                 
    #optimizer.zero_grad() #                                                                                                                                                                                                               
    out = model(data.x, data.edge_index, data.batch).to(device)  # Perform a single forward pass                                                                                                                                          
    out = out.cpu()
    #print(out) 
    #print(data.elc) 

   # neww = []
    #for j in range(len(data)):
        #neww.append(out[j].item() -data.elc[j][0])
     #   neww.append(out[j].item() -data.elc[j])
    #outt+=[[neww]]
    #outt+=[[out[0][0].item()-data.elc[0]]] 
    outt+=[[out[0][0].item()-data.elc[0]]]
    for j in range(0,int(len(data.imass))): 
        pi_mas.append(data.imass[j] ) 
#return outt  
#b= print_outbb() 
b=outt 


pi_x = np.zeros(len(outt))
for i in range(len(outt)):
    pi_x[i] = outt[i][0]
pi_yy = [] 
#p_yy = [] 
#pi_x = outt
pi_rec_y = []
#p_rec_y = []
for i, data in enumerate(dataset):
    data = data#.to(device) #put to GPU
    #break
    jj= 0
    for j in range(0,int(len(data.y))): 
        pi_yy.append(data.y[j].item() -data.elc[j] ) 
      
    for j in range(len(data.rec)):
        pi_rec_y.append(data.rec[j]-data.elc[j])
       # p_rec_y.append(data.rec[j][0][1])
       
pi_yy = np.array(pi_yy)
pi_rec_y = np.array(pi_rec_y)
plt.figure(10)

plt.scatter(pi_x, pi_rec_y, color = 'blue')
plt.title('Correlation Between GNN and REC') 
#plt.xlim((-10,15)) 
plt.ylabel('REC Vertex (cm)') 
plt.xlabel('GNN Vertex (cm)') 

plt.figure(11)

plt.scatter(pi_x, pi_yy, color = 'green')
plt.title('Correlation Between GNN and Truth') 
#plt.xlim((-10,15)) 
plt.ylabel('MC Vertex (cm)') 
plt.xlabel('GNN Vertex (cm)') 

plt.figure(12) 

plt.scatter(pi_rec_y, pi_yy, color = 'green')
plt.title('Correlation Between REC and Truth') 
#plt.xlim((-10,15)) 
plt.ylabel('MC Vertex (cm)') 
plt.xlabel('REC Vertex (cm)')  


plt.figure(13)
plt.hist2d(pi_rec_y, pi_yy, bins=50, cmap='viridis')
plt.colorbar(label='Density')
plt.title('Correlation Between REC and Truth')
plt.ylabel('MC Vertex (cm)')
plt.xlabel('REC Vertex (cm)')

plt.figure(14)
plt.hist2d(pi_x, pi_yy, bins=50, cmap='viridis')
plt.colorbar(label='Density')
plt.title('Correlation Between GNN and Truth')
plt.ylabel('MC Vertex (cm)')
plt.xlabel('GNN Vertex (cm)')

plt.figure(15)
plt.hist2d(pi_x, pi_rec_y, bins=50, cmap='viridis')
plt.colorbar(label='Density')
plt.title('Correlation Between GNN and Truth')
plt.ylabel('REC Vertex (cm)')
plt.xlabel('GNN Vertex (cm)')

from mpl_toolkits.mplot3d import Axes3D
from scipy.stats import gaussian_kde
xyz = np.vstack([pi_x, pi_yy, pi_rec_y])
density = gaussian_kde(xyz)(xyz)

# Create the 3D scatter plot
fig = plt.figure(16)
ax = fig.add_subplot(111, projection='3d')
sc = ax.scatter(pi_x, pi_yy, pi_rec_y, c=density, cmap='viridis')
ax.set_xlim(-50, 150)
ax.set_ylim(-50, 150)
ax.set_zlim(-150, 150)

# Add color bar which maps values to colors
cbar = plt.colorbar(sc)
cbar.set_label('Density')

# Create the 3D scatter plot
fig = plt.figure(17)
ax = fig.add_subplot(111, projection='3d')
sc = ax.scatter(pi_x, pi_yy, pi_rec_y, c=density, cmap='viridis')
ax.set_xlim(-50, 75)
ax.set_ylim(-50, 150)
ax.set_zlim(-150, 150)

# Add color bar which maps values to colors
cbar = plt.colorbar(sc)
cbar.set_label('Density')

for iiii in range(18):  
    plt.figure(iiii)
    idddddk = '800_finall'+str(iiii)+'.png'
    #plt.savefig(idddddk)   
print('doneeeeeee')
np.save('s_to_b_valuesHnrecsingle1100.npy', s_to_b_values) 
np.save('purity_valuesHnrecsingle1100.npy', purity_values) 
np.save('efficiency_valuesHnrecsingle1100.npy', efficiency_values)

PATH = '/hpc/group/vossenlab/kam264/800_model_testt'
torch.save(model.state_dict(), PATH)
print('done with test')'''      

f = open("/hpc/group/vossenlab/kam264/dir_name_2/batch_filer.py", "w")  
f.write(code)              
f.close()         


In [ ]:
15498747, idddddk = 'letsgoo'+str(iiii)+'.png', single dataset, 200 epochs. 
15553908, testing for scatter. 15554469, for all the sactters. 
new plots, 500 single 15556937, plotsingle#.png
15556977, new plots, four, 60 each, plotsfour#.png 
15653361, quikc test for saving. 

15653435 , 800 single, new grpahss, single_

16012885, final. 

In [15]:
import os   
import numpy as np 
import numpy.ma as ma   
#import awkward as ak 
from tqdm import tqdm 
import torch 
import torch_geometric as tg  
import torch_geometric 
from torch_geometric.data import Data 
#import torch 
from torch_geometric.data import InMemoryDataset, download_url  
import torch_geometric.transforms as T 

#NOTE: NEW 2/20/23      
from typing import List, Union     

from torch_geometric.data import Data, HeteroData 
from torch_geometric.data.datapipes import functional_transform
from torch_geometric.transforms import BaseTransform       
torch.cuda.empty_cache()  
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import GraphConv
from torch_geometric.nn import global_mean_pool
from torch_geometric.nn.norm import GraphNorm, BatchNorm 
from torch.utils.data import random_split 
from torch_geometric.loader import DataLoader 

class MyOwnDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None, pre_filter=None):
        super().__init__(root, transform, pre_transform, pre_filter)
        self.load(self.processed_paths[0])
        # For PyG<2.4:
        # self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return ['some_file_1', 'some_file_2']

    @property
    def processed_file_names(self):
        return ['data.pt']

    def process(self):
        # Read data into huge `Data` list.
        data_list = None

        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]

        self.save(data_list, self.processed_paths[0]) 
root = '/hpc/group/vossenlab/kam264/Hmom4' 
#root = '/hpc/group/vossenlab/kam264/QuatorL_Last'
dataset = MyOwnDataset(
        root,
        transform=None, #T.Compose([T.ToUndirected(),T.KNNGraph(k=6)]),
        pre_transform=None,
        pre_filter=None
    )    

pi_momentum  = []
for i,data in enumerate(dataset):
    data = data#.to(device) #switch to GPU                                                                                                                                                                                                 
    #optimizer.zero_grad() #                                                                                                                                                                                                               
    #out = model(data.x, data.edge_index, data.batch).to(device)  # Perform a single forward pass                                                                                                                                          
    #out = out.cpu()
    #print(out) 
    #print(data.elc)  
   # neww = []
    #for j in range(len(data)):
        #neww.append(out[j].item() -data.elc[j][0])
     #   neww.append(out[j].item() -data.elc[j])
    #outt+=[[neww]]
    #outt+=[[out[0][0].item()-data.elc[0]]] 
    #outt+=[[out[0][0].item()-data.elc[0]]]
    for j in range(0,int(len(data.momentumt))): 
        pi_momentum.append(data.momentumt[j] ) 
pi_momentum = np.array(pi_momentum) 
j=2 
indices = np.where(np.logical_and(pi_momentum < 0+j/2, pi_momentum > 0+j/2-.5) )
print(indices)  

(array([    0,     1,     2, ..., 18629, 18632, 18634]),)


In [ ]:
15427603 ,  15427911, 15428311 

In [ ]:
plt.figure(10) #############################################################
pi_yy = np.array(pi_yy)
pi_x = np.array(pi_x)
pi_rec_y = np.array(pi_rec_y)#np.logical_and
indices_two = np.where(np.logical_and(pi_yy < -1.5, pi_yy > -2) ) 
#indices_two = np.where(pi_yy <-1.5 and pi_yy>-2 )
#print(indices_two)
#print(pi_x[indices_two])  

diff_x = [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
diff_rec  = [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
for j in range(1,24):
    indices = np.where(np.logical_and(pi_yy < -2+j/2, pi_yy > -2+j/2-.5) ) 
    dif_x =[ pi_x[indices]]
    dif_rec = [pi_rec_y[indices]]
    diff_x[j-1].append(dif_x)
    diff_rec[j-1].append(dif_rec)
mean_x = np.zeros(24)
mean_rec = np.zeros(24)
error_x = np.zeros(24)
error_rec = np.zeros(24)
for i in range(24): 
    mean_x[i] = np.nanmean(diff_x[i])
    mean_rec[i] = np.nanmean(diff_rec[i]) 
    error_x[i] = np.nanstd(diff_x[i])
    error_rec[i] = np.nanstd(diff_rec[i])
xxx = np.linspace(-2,10,24)
print('biined distribution mean. ')
print(mean_x)
print(mean_rec)
print(error_x)
print(error_rec)
plt.scatter(xxx, mean_x)
plt.scatter(xxx, mean_rec, alpha = .5 )  
plt.errorbar(xxx, mean_x,  yerr=error_x, fmt="o")   
plt.errorbar(xxx, mean_rec,  yerr=error_rec, fmt="o", alpha = .5)
plt.title('binned distibution of pion vertex based on true vertex, orange is rec, blue is GNN')
plt.xlabel('Pion vertex (cm)') 
plt.ylabel('abolute error from True (cm)')
#plt.savefig('Newstuff4.png')   ##############################################

In [7]:
#basecode what I showed on Monday.  , go to end for new code. 
code = '''import os   
import numpy as np
import numpy.ma as ma   
#import awkward as ak 
from tqdm import tqdm 
import torch 
import torch_geometric as tg  
import torch_geometric 
from torch_geometric.data import Data 
#import torch 
from torch_geometric.data import InMemoryDataset, download_url  
import torch_geometric.transforms as T 

#NOTE: NEW 2/20/23      
from typing import List, Union     

from torch_geometric.data import Data, HeteroData 
from torch_geometric.data.datapipes import functional_transform
from torch_geometric.transforms import BaseTransform       
torch.cuda.empty_cache()  
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import GraphConv
from torch_geometric.nn import global_mean_pool
from torch_geometric.nn.norm import GraphNorm, BatchNorm 
from torch.utils.data import random_split 
from torch_geometric.loader import DataLoader 

class MyOwnDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None, pre_filter=None):
        super().__init__(root, transform, pre_transform, pre_filter)
        self.load(self.processed_paths[0])
        # For PyG<2.4:
        # self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return ['some_file_1', 'some_file_2']

    @property
    def processed_file_names(self):
        return ['data.pt']

    def process(self):
        # Read data into huge `Data` list.
        data_list = None

        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]

        self.save(data_list, self.processed_paths[0]) 

#root = '/hpc/group/vossenlab/kam264/L_imass3'
#root = '/hpc/group/vossenlab/kam264/onlylambda_25k' 
#root = '/hpc/group/vossenlab/kam264/Lambda_13_16'
#root = '/hpc/group/vossenlab/kam264/Lambda_1all' 

batch_size = 1 
LR =1e-2
torch.cuda.empty_cache()   

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels): 
        super(GCN, self).__init__()
#         torch.manual_seed(12345)
        self.conv1 = GCNConv(in_channels, hidden_channels)#.jittable() #NOTE: NEEDED FOR DEPLOYMENT IN CMAKE
        self.conv2 = GCNConv(hidden_channels, hidden_channels)#.jittable()
        #self.block2 = nn.DataParallel(self.block2)
        #self.conv2 = torch.nn.DataParallel(self.conv2) #this was trying the parallization thing. 
        self.conv3 = GCNConv(hidden_channels, hidden_channels)#.jittable()
        #self.conv3 = torch.nn.DataParallel(self.conv3)
        self.lin1 = Linear(hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, hidden_channels)
        self.lin3 = Linear(hidden_channels, out_channels)
        self.bn1 = torch_geometric.nn.norm.GraphNorm(hidden_channels)
        self.bn2 = torch_geometric.nn.norm.GraphNorm(hidden_channels)
        self.bn3 = torch_geometric.nn.norm.GraphNorm(hidden_channels)

    def forward(self, x, edge_index, batch):  
        
        x = self.conv1(x, edge_index) #input layer                             
                                                      
        x = self.bn1(x) #normalize it                                          

        x = x.relu() #activation                                               
        x = self.conv2(x, edge_index)
        x = self.bn2(x)
        x = x.relu()
#         print("x.relu() = ",x)  
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv3(x, edge_index)
        x = self.bn3(x)
#         # 2. Readout layer                                                   
        x = global_mean_pool(x, batch)
        # 3. Apply a final classifier                                          
        x = F.dropout(x, p=0.5, training=self.training) #for overfittin        
        x = self.lin3(x) 
 
        return x
def RMSELoss(out,y):
    return torch.sqrt(torch.mean((out-y)**2))
def train():
    model.train() #initailize the model                                                                                                                                                                                                        
    #for i, data in tqdm(enumerate(train_loader)): #perhaps tqdm(enumerate(train_loader)), i is index, data jsut moves through all the dtaa in trainingg                                                                                      
    for i,data in enumerate(train_loader):
        data = data.to(device) #switch to GPU                                                                                                                                                                                                 
        optimizer.zero_grad() #                                                                                                                                                                                                               
        out = model(data.x, data.edge_index, data.batch).to(device)  # Perform a single forward pass                                                                                                                                          
        yy = []
        for j in range(0,len(out)):
            yy+= [[data.y[j].item()]]

        yy = torch.tensor(yy).to(device) 
        #print(out)
        #print(yy)
        loss = losss(out, yy).to(device) #compute the loss  
        #print(loss)
        loss.backward() #get the gradients.                                                                                                                                                                                                   
        optimizer.step() 
def test(loader): 
    length = len(loader.dataset)
    model.eval() #evaluate teh model.                                                                                                                                                                                                         

    #mse_tot = []                                                                                                                                                                                                                             
    mse_total = 0
    mse_pi = 0
    mse_p = 0
    #r                                                                                                                                                                                                                                        
    #for data in tqdm(loader):  # Iterate in batches over the training/test dataset.                                                                                                                                                          
    for data in loader:
        data = data.to(device) #put to GPU                                                                                                                                                                                                    
        out = model(data.x, data.edge_index, data.batch).to(device) #evalueate                                                                                                                                                                
        #this and the for loop is converting data.y to a tensor in the same shape as out rows and 2 columns first is y_pion second is y_proton                                                                                                
        yy = []
        for j in range(0,len(out)):
            yy+= [[data.y[j].item()]]
        yy = torch.tensor(yy).to(device) 
        loss = losss(out, yy).cpu() #getting teh loss function                                                                                                                                                                                
        mse_total+=loss.item() #getting the mse (total)                                                                                                                                                                                       

    return mse_total/length 
def print_out():
    model.eval() #initailize the model                                                                                                                                                                                                        
    #for i, data in tqdm(enumerate(train_loader)): #perhaps tqdm(enumerate(train_loader)), i is index, data jsut moves through all the dtaa in trainingg                                                                                      
    outt= []
    for i,data in enumerate(test_loader):
        data = data.to(device) #switch to GPU                                                                                                                                                                                                 
        optimizer.zero_grad() #                                                                                                                                                                                                               
        out = model(data.x, data.edge_index, data.batch).to(device)  # Perform a single forward pass                                                                                                                                          
        out = out.cpu()

        outt+=[[out.detach().numpy()]]
    return outt

def print_outb():
    model.eval() #initailize the model                                                                                                                                                                                                        
    #for i, data in tqdm(enumerate(train_loader)): #perhaps tqdm(enumerate(train_loader)), i is index, data jsut moves through all the dtaa in trainingg      

    outt= [] 
    #for i,data in enumerate(dataset):
    for i,data in enumerate(test_loader):
        data = data.to(device) #switch to GPU                                                                                                                                                                                                 
        optimizer.zero_grad() #                                                                                                                                                                                                               
        out = model(data.x, data.edge_index, data.batch).to(device)  # Perform a single forward pass                                                                                                                                          
        out = out.cpu()

        neww = []
        for j in range(len(data)):
            neww.append(out[j].item() -data.elc[j][0]) 
        outt+=[[neww]]
    return outt  
#model = GCN(dataset.num_node_features,64,2)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') 
#devicee = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu') 
print("Device = ",device) 

print("DEBUGGING: torch.cuda.is_available() = ",torch.cuda.is_available())
train_metrics = {'mse':[], "rmse":[] } 
vall_metrics = {'mse':[], "rmse":[] }

for iii in range(1):   
    if iii>0: 
        #del dataset
        del train_loader  
        del val_loader 
    if iii==0:
        root = '/hpc/group/vossenlab/kam264/80k_neww'   
    dataset = MyOwnDataset(
            root,
            transform=None, #T.Compose([T.ToUndirected(),T.KNNGraph(k=6)]),
            pre_transform=None,
            pre_filter=None
        ) 
    dataset
    model = GCN(dataset.num_node_features,64,2)
    fracs = [0.9, 0.08, 0.02] #percent of dataset used for training testing and validatoin 80%,10%,10% #NOTE: SHOULD CHECK np.sum(fracs) == 1 and len(fracs)==3
    fracs = [torch.sum(torch.tensor(fracs[:idx])) for idx in range(1,len(fracs)+1)] #get the indexes for training ... parts to use. 
    #print(fracs)
    split1, split2 = [int(len(dataset)*frac) for frac in fracs[:-1]] 
    train_dataset = dataset[:split1]
    val_dataset = dataset[split1:split2]
    test_dataset = dataset[split2:]   
     
    print(f'Number of training graphs: {len(train_dataset)}')
    print(f'Number of validation graphs: {len(val_dataset)}') 

    from torch_geometric.loader import DataLoader 
    #from torch.utils.data import WeightedRandomSampler

    train_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=8, shuffle=True)#, drop_last=True)
    val_loader = DataLoader(val_dataset,  batch_size=batch_size,num_workers=8,  shuffle=True)
    model = GCN(dataset.num_node_features, dataset.num_classes, 1).to(device) #initiate the model, #2 is the number of outputs here is 2 as pion_z, proton_z 
    model = model.to(device)  
    del dataset
    del train_dataset
    del val_dataset
    del test_dataset 
    optimizer = torch.optim.Adam(model.parameters(), lr= LR)  
    
    losss = RMSELoss  
    nepochs = 1      
    
    for epoch in range(nepochs):  
  
        #print("BEFORE TRAIN()")                                                                                                                                                                                                                  
        train() 
        #print("BEFORE TEST(TRAIN_LOADER)")                                                                                                                                                                                                       
        #train_mse, train_rmse, train_mse_pi, train_rmse_pi, train_mse_p, train_rmse_p = test(train_loader)
        train_mse = test(train_loader)

        train_metrics['mse'].append(train_mse) 

        vall_mse =test(val_loader) 


        vall_metrics['mse'].append(vall_mse) 

        if epoch%9==0:
            print("Epoch ",epoch," Train mse: ",train_mse)
            print("Epoch ",epoch," Validation mse: ",vall_mse)
        if epoch==(nepochs-1):
            #PATH = '/work/clas12/users/mfmce/CLAS12_Lambda_resolution_REU_2023/model_best_auc.pt' 

            #a = print_out()
            #b=  print_outb() 
            print("Epoch ",epoch," Train mse: ",train_mse)
            print("Epoch ",epoch," Validation mse: ",vall_mse)

root = '/hpc/group/vossenlab/kam264/Hmom4' 
#root = '/hpc/group/vossenlab/kam264/QuatorL_Last'
dataset = MyOwnDataset(
        root,
        transform=None, #T.Compose([T.ToUndirected(),T.KNNGraph(k=6)]),
        pre_transform=None,
        pre_filter=None
    )    

model.eval() #initailize the model                                                                                                                                                                                                        
#for i, data in tqdm(enumerate(train_loader)): #perhaps tqdm(enumerate(train_loader)), i is index, data jsut moves through all the dtaa in trainingg      

outt= []  
pi_mas = []
#for i,data in enumerate(dataset): 
for i,data in enumerate(dataset):
    data = data.to(device) #switch to GPU                                                                                                                                                                                                 
    #optimizer.zero_grad() #                                                                                                                                                                                                               
    out = model(data.x, data.edge_index, data.batch).to(device)  # Perform a single forward pass                                                                                                                                          
    out = out.cpu()
    #print(out) 
    #print(data.elc) 

   # neww = []
    #for j in range(len(data)):
        #neww.append(out[j].item() -data.elc[j][0])
     #   neww.append(out[j].item() -data.elc[j])
    #outt+=[[neww]]
    #outt+=[[out[0][0].item()-data.elc[0]]] 
    outt+=[[out[0][0].item()-data.elc[0]]]
    for j in range(0,int(len(data.imass))): 
        pi_mas.append(data.imass[j] ) 
#return outt  
#b= print_outbb() 
b=outt 

import matplotlib.pyplot as plt
pi_x = np.zeros(len(outt))
for i in range(len(outt)):
    pi_x[i] = outt[i][0]
pi_yy = [] 
#p_yy = [] 
#pi_x = outt
pi_rec_y = []
#p_rec_y = []
for i, data in enumerate(dataset):
    data = data#.to(device) #put to GPU
    #break
    jj= 0
    for j in range(0,int(len(data.y))): 
        pi_yy.append(data.y[j].item() -data.elc[j] ) 
      
    for j in range(len(data.rec)):
        pi_rec_y.append(data.rec[j]-data.elc[j])
       # p_rec_y.append(data.rec[j][0][1])

#plott_pi = np.zeros((len(pi_x),4))
#plott_p = np.zeros((len(p_x),2)) 
plott_pi = np.zeros((len(pi_x),3))
for i in range(len(pi_x)):
#for i in range(982):
    plott_pi[i][0] = pi_yy[i]
    plott_pi[i][1] = pi_x[i]# [0]
    #plott_pi[i][3] = pi_val[i]
   # plott_pi[i][2] = pi_rec_y[i] 
   # plott_p[i][0] = p_yy[i]
   # plott_p[i][1] = p_x[i]
    #plott_p[i][2]  = p_rec_y[i] 
for i in range(len(pi_x)): 
    plott_pi[i][2] = pi_rec_y[i] 
#plott_pi 
plt.figure(0) ##################
plt.title('location of Pion vertex') 
#plt.axvline(x = np.mean(pi_x), color = 'blue') 
#plt.axvline(x = np.mean(pi_yy), color = 'orange') 
plt.hist(x = plott_pi, histtype ='step', color = ['orange', 'blue', 'green'], bins = 500 ) 
#plt.hist(x = plott_pi, histtype ='step', bins = 500 )
#plt.legend(['Predicted', 'True', 'REC']) 
#plt.legend(['True', 'Predicted', 'REC']) 
plt.xlim((-10,15)) 
plt.ylabel('count') 
plt.xlabel('Location of Pion vertex (cm)')  
 #############################################################

plt.figure(1) #######################
pi_yy = np.array(pi_yy)
pi_x = np.array(pi_x)
pi_rec_y = np.array(pi_rec_y)#np.logical_and
indices_two = np.where(np.logical_and(pi_yy < -1.5, pi_yy > -2) ) 
#indices_two = np.where(pi_yy <-1.5 and pi_yy>-2 )
#print(indices_two)
#print(pi_x[indices_two])  

diff_x = [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
diff_rec  = [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
for j in range(1,24):
    indices = np.where(np.logical_and(pi_yy < -2+j/2, pi_yy > -2+j/2-.5) ) 
    dif_x =[ abs(pi_x[indices] - pi_yy[indices])]
    dif_rec = [abs(pi_rec_y[indices] - pi_yy[indices])]
    diff_x[j-1].append(dif_x)
    diff_rec[j-1].append(dif_rec)
mean_x = np.zeros(24)
mean_rec = np.zeros(24)
error_x = np.zeros(24)
error_rec = np.zeros(24)
for i in range(24): 
    mean_x[i] = np.nanmean(diff_x[i])
    mean_rec[i] = np.nanmean(diff_rec[i]) 
    error_x[i] = np.nanstd(diff_x[i])
    error_rec[i] = np.nanstd(diff_rec[i])
xxx = np.linspace(-2,10,24)
plt.scatter(xxx, mean_x)
plt.scatter(xxx, mean_rec, alpha = .5 )  
plt.errorbar(xxx, mean_x,  yerr=error_x, fmt="o")   
plt.errorbar(xxx, mean_rec,  yerr=error_rec, fmt="o", alpha = .5)
plt.title('absolute error based on pion vertex, orange is rec, blue is GNN')
plt.xlabel('Pion vertex (cm)') 
plt.ylabel('abolute error from True (cm)')
 #############################################################
plt.figure(2) ####################### 
diff_x = [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
diff_rec  = [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
for j in range(1,24):
    indices = np.where(np.logical_and(pi_yy < -2+j/2, pi_yy > -2+j/2-.5) ) 
    dif_x =[ (pi_x[indices] - pi_yy[indices])]
    dif_rec = [(pi_rec_y[indices] - pi_yy[indices])]
    diff_x[j-1].append(dif_x)
    diff_rec[j-1].append(dif_rec)
mean_x = np.zeros(24)
mean_rec = np.zeros(24)
error_x = np.zeros(24)
error_rec = np.zeros(24)
for i in range(24): 
    mean_x[i] = np.nanmean(diff_x[i])
    mean_rec[i] = np.nanmean(diff_rec[i]) 
    error_x[i] = np.nanstd(diff_x[i])
    error_rec[i] = np.nanstd(diff_rec[i])
xxx = np.linspace(-2,10,24)
plt.scatter(xxx, mean_x)
plt.scatter(xxx, mean_rec, alpha = .5 )  
plt.errorbar(xxx, mean_x,  yerr=error_x, fmt="o")   
plt.errorbar(xxx, mean_rec,  yerr=error_rec, fmt="o", alpha = .5)
plt.title(' error based on pion vertex, orange is rec, blue is GNN')
plt.xlabel('Pion vertex (cm)') 
plt.ylabel(' error from True (cm)')
 #############################################################

pi_mas = []
pi_LamT = []
#p_rec_y = []
for i, data in enumerate(dataset):
    data = data#.to(device) #put to GPU
    #break
    #print(data.LamT)
    for j in range(0,int(len(data.imass))): 
        pi_mas.append(data.imass[j]) 
        pi_LamT.append(data.LamT[j]) 
Total_signal = pi_LamT.count(1)
print('S/B')
print('Purity') 
print('Efficienty')
s_to_b_values = []
purity_values = []
efficiency_values = []
for j in range(0, 20):
    pi_LamTt = []
    #imass
    print('cut ',j/2)
    for i in range(len(pi_x)): 
        if pi_x[i]>j/2:
            #imass.append(pi_mas[i])
            pi_LamTt.append(pi_LamT[i])
    print(pi_LamTt.count(1))
    #print(pi_LamT.count(0))
    print(pi_LamTt.count(1)/pi_LamTt.count(0))
    print(pi_LamTt.count(1)/(len(pi_LamTt)))
    print(pi_LamTt.count(1)/(Total_signal))
    #print(pi_LamTt.count(1)/len(pi_LamTt))
    s_to_b = pi_LamTt.count(1)/pi_LamTt.count(0)
    purity = pi_LamTt.count(1)/(len(pi_LamTt))
    efficiency = pi_LamTt.count(1)/(Total_signal)
    s_to_b_values.append(s_to_b)
    purity_values.append(purity)
    efficiency_values.append(efficiency)
    print()
    plt.figure(3)
    plt.scatter(j/2, pi_LamTt.count(1)/pi_LamTt.count(0), color = 'blue')
    plt.xlabel('pion vetex cut')
    plt.ylabel('S/B') 
    plt.title('Graph of S/B as pion vertex cuts')
    
    plt.figure(4)
    plt.scatter(j/2, pi_LamTt.count(1)/(len(pi_LamTt)), color = 'red')
    plt.xlabel('pion vetex cut')
    plt.ylabel('Purity') 
    plt.title('Graph of Purity as pion vertex cuts')
                
    plt.figure(5)
    plt.scatter(j/2, pi_LamTt.count(1)/(Total_signal), color = 'purple') 
    plt.xlabel('pion vetex cut')
    plt.ylabel('Efficiency ') 
    plt.title('Graph of Efficiency  as pion vertex cuts')
    #print(len(pi_LamT))
    #plt.figure()
    #plt.title('cut'+str( j/2))  
    #plt.hist(imass, bins = 25)
##################################################################new stufff. 
pi_x = np.zeros(len(outt)) 
for i in range(len(outt)):
    pi_x[i] = outt[i][0]
pi_yy = [] 
#p_yy = [] 
#pi_x = outt
pi_rec_y = []
#p_rec_y = []
for i, data in enumerate(dataset):
    data = data#.to(device) #put to GPU
    jj= 0
    for j in range(0,int(len(data.y))): 
        pi_yy.append(data.y[j].item() -data.elc[j] ) 
      
    for j in range(len(data.rec)):
        pi_rec_y.append(data.rec[j]-data.elc[j])

plott_pi = np.zeros((len(pi_x),3))
for i in range(len(pi_x)):
    if pi_rec_y[i]>4:
        if pi_rec_y[i]<6:
            plott_pi[i][0] = pi_yy[i]
            plott_pi[i][1] = pi_x[i]# [0]

for i in range(len(pi_x)): 
    if pi_rec_y[i]>4:
        if pi_rec_y[i]<6:
            plott_pi[i][2] = pi_rec_y[i] 
#plott_pi 

plt.figure(7) #############################################################
pi_yy = np.array(pi_yy)
pi_x = np.array(pi_x)
pi_rec_y = np.array(pi_rec_y)#np.logical_and
indices_two = np.where(np.logical_and(pi_yy < -1.5, pi_yy > -2) ) 
#indices_two = np.where(pi_yy <-1.5 and pi_yy>-2 )
#print(indices_two)
#print(pi_x[indices_two])  

diff_x = [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
diff_rec  = [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
for j in range(1,24):
    indices = np.where(np.logical_and(pi_rec_y < -2+j/2, pi_rec_y > -2+j/2-.5) ) 
    dif_x =[ abs(pi_x[indices] - pi_yy[indices])]
    dif_rec = [abs(pi_rec_y[indices] - pi_yy[indices])]
    diff_x[j-1].append(dif_x)
    diff_rec[j-1].append(dif_rec)
mean_x = np.zeros(24)
mean_rec = np.zeros(24)
error_x = np.zeros(24)
error_rec = np.zeros(24)
for i in range(24): 
    mean_x[i] = np.nanmean(diff_x[i])
    mean_rec[i] = np.nanmean(diff_rec[i]) 
    error_x[i] = np.nanstd(diff_x[i])
    error_rec[i] = np.nanstd(diff_rec[i])
xxx = np.linspace(-2,10,24)
print('based on rec')
print(mean_x)
print(mean_rec)
print(error_x)
print(error_rec)
print()
plt.scatter(xxx, mean_x)
plt.scatter(xxx, mean_rec, alpha = .5 )  
plt.errorbar(xxx, mean_x,  yerr=error_x, fmt="o")   
plt.errorbar(xxx, mean_rec,  yerr=error_rec, fmt="o", alpha = .5)
plt.title('Error Based on rec Vertex')
plt.xlabel('Pion rec Vertex (cm)') 
plt.ylabel('Abolute Error from True (cm)')
#plt.savefig('Newstuff2.png')

plt.figure(8) #############################################################
pi_yy = np.array(pi_yy)
pi_x = np.array(pi_x)
pi_rec_y = np.array(pi_rec_y)#np.logical_and
indices_two = np.where(np.logical_and(pi_yy < -1.5, pi_yy > -2) ) 
#indices_two = np.where(pi_yy <-1.5 and pi_yy>-2 )
#print(indices_two)
#print(pi_x[indices_two])  

diff_x = [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
diff_rec  = [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
for j in range(1,24):
    indices = np.where(np.logical_and(pi_x < -2+j/2, pi_x > -2+j/2-.5) ) 
    dif_x =[ abs(pi_x[indices] - pi_yy[indices])]
    dif_rec = [abs(pi_rec_y[indices] - pi_yy[indices])]
    diff_x[j-1].append(dif_x)
    diff_rec[j-1].append(dif_rec)
mean_x = np.zeros(24)
mean_rec = np.zeros(24)
error_x = np.zeros(24)
error_rec = np.zeros(24)
for i in range(24): 
    mean_x[i] = np.nanmean(diff_x[i])
    mean_rec[i] = np.nanmean(diff_rec[i]) 
    error_x[i] = np.nanstd(diff_x[i])
    error_rec[i] = np.nanstd(diff_rec[i])
xxx = np.linspace(-2,10,24)
print('based on GNN')
print(mean_x)
print(mean_rec)
print(error_x)
print(error_rec)
print() 
plt.scatter(xxx, mean_x)
plt.scatter(xxx, mean_rec, alpha = .5 )  
plt.errorbar(xxx, mean_x,  yerr=error_x, fmt="o")   
plt.errorbar(xxx, mean_rec,  yerr=error_rec, fmt="o", alpha = .5)
plt.title('Error based on GNN vertex (abs)')
plt.xlabel('Pion GNN vertex (cm)') 
plt.ylabel('Abolute Error from True (cm)')
#plt.savefig('Newstuff3.png') 







outt= [] 
pi_momentum  = []
#for i,data in enumerate(dataset): 
for i,data in enumerate(dataset):
    data = data.to(device) #switch to GPU                                                                                                                                                                                                 
    #optimizer.zero_grad() #                                                                                                                                                                                                               
    #out = model(data.x, data.edge_index, data.batch).to(device)  # Perform a single forward pass                                                                                                                                          
    #out = out.cpu()
    #print(out) 
    #print(data.elc)  
   # neww = []
    #for j in range(len(data)):
        #neww.append(out[j].item() -data.elc[j][0])
     #   neww.append(out[j].item() -data.elc[j])
    #outt+=[[neww]]
    #outt+=[[out[0][0].item()-data.elc[0]]] 
    #outt+=[[out[0][0].item()-data.elc[0]]]
    for j in range(0,int(len(data.momentumt))): 
        pi_momentum.append(data.momentumt[j] )  
        
pi_momentum = np.array(pi_momentum) 
plt.figure(9) ####################### 
diff_x = [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
diff_rec  = [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
for j in range(1,20):
    indices = np.where(np.logical_and(pi_momentum < 0+j/4, pi_momentum > 0+j/4-.25) )  
    dif_x =[ abs(pi_x[indices] - pi_yy[indices])]
    dif_rec = [abs(pi_rec_y[indices] - pi_yy[indices])]
    diff_x[j-1].append(dif_x)
    diff_rec[j-1].append(dif_rec)
mean_x = np.zeros(20)
mean_rec = np.zeros(20)  
error_x = np.zeros(20)
error_rec = np.zeros(20)
for i in range(20): 
    mean_x[i] = np.nanmean(diff_x[i])
    mean_rec[i] = np.nanmean(diff_rec[i]) 
    error_x[i] = np.nanstd(diff_x[i])
    error_rec[i] = np.nanstd(diff_rec[i])
xxx = np.linspace(0,5,20) 
plt.scatter(xxx, mean_x)
plt.scatter(xxx, mean_rec, alpha = .5 )  
plt.errorbar(xxx, mean_x,  yerr=error_x, fmt="o")   
plt.errorbar(xxx, mean_rec,  yerr=error_rec, fmt="o", alpha = .5)
plt.title(' Error based on Momentum')
plt.xlabel('Pion momentum (cm)') 
plt.ylabel(' Error from True (cm)')  



for iiii in range(10):  
    plt.figure(iiii)
    idddddk = 'big_test'+str(iiii)+'.png'
    plt.savefig(idddddk)    
print('doneeeeeee')
np.save('s_to_b_valuesHnrecsingle1100.npy', s_to_b_values)
np.save('purity_valuesHnrecsingle1100.npy', purity_values) 
np.save('efficiency_valuesHnrecsingle1100.npy', efficiency_values)'''      

f = open("/hpc/group/vossenlab/kam264/dir_name_2/batch_filer.py", "w")   
#f = open("/hpc/group/vossenlab/mfm45/forkeegan/batch_filerr.py", "w")  
f.write(code)              
f.close()          

In [ ]:
 15500514